In [ ]:
import pandas as pd
import numpy as np
import os
from utils import *
import cvxpy as cp
from datetime import timedelta

def get_index_price(index:str, start_date:str, end_date:str, field:str='close'):
    index_list = ['000300.SH', '000905.SH', '000852.SH', '000985.SH']
    if index not in index_list:
        print(f'pleace select from {index_list}')
    
 
    return index.pivot(index='trade_date', columns='ts_code', values=field)

def port_opt(period_list, industry_list, barra_list, past_weight, stock_today, index_today, barra_limit, turnover=0.15):
    # 定义要优化的变量，设置非负约束
    weights = cp.Variable(shape=(len(stock_today), 1), nonneg=True)

    # 收集约束条件
    constraints = []

    # 1. 权重之和等于1
    constraints.append(cp.sum(weights) == 1)

    # 2. 个股权重限制为1%
    constraints.append(weights <= 0.01)

    # 3. 行业权重限制与指数的差异限制为5%
    constraints.append(weights[:, 0] @ stock_today[industry_list] - index_today[industry_list] <= 0.05)
    constraints.append(weights[:, 0] @ stock_today[industry_list] - index_today[industry_list] >= -0.05)

    # 4. barra 偏离限制
    for _barra in barra_list:
        constraints.append(weights[:, 0] @ stock_today[_barra] - index_today[_barra] <= barra_limit)
        constraints.append(weights[:, 0] @ stock_today[_barra] - index_today[_barra] >= -barra_limit)

    # 5. 限制换手率（权重变化的总和）为15%
    trade_cost_sum = cp.sum(cp.abs(weights - past_weight[['past_weight']].iloc[:len(stock_today)]))

    if past_weight['past_weight'].sum() > 0.5:  # 首日排除
        constraints.append(trade_cost_sum <= turnover)  # 单日换手不超过15%

    # 定义目标函数
    profit_sum = cp.sum(cp.multiply(weights, stock_today[period_list])) 
    obj = profit_sum 

    # 传递问题给求解器
    problem = cp.Problem(cp.Maximize(obj), constraints)

    try:
        problem.solve(solver='ECOS', qcp=True, max_iters=2000)
    except:
        problem.solve()

    weights_value = weights.value
    weights_value = pd.concat([stock_today[['Symbol']].reset_index(drop=True),
                                pd.DataFrame(weights_value[:, 0], columns=['weight']).reset_index(drop=True)],
                                axis=1)
    weights_value.loc[weights_value['weight'] < 0.00001, 'weight'] = 0
    weights_value = weights_value[weights_value['weight'] != 0]
    weights_value['weight'] = weights_value['weight'] / weights_value['weight'].sum()
    weights_value = pd.merge(left=weights_value, right=past_weight[['Symbol']], on='Symbol', how='outer')
    weights_value.fillna(0, inplace=True)
    weights_value = weights_value.set_index('Symbol')['weight'].to_dict()
    return weights_value, problem.status

def trade_cal(past_weight, holding_df, trade_target, today_stock_return, sell_trade_cost = 0.0, buy_trade_cost = 0.0, drop_list=[]):

    weight_diff = pd.merge(left=past_weight, right=trade_target, on='Symbol', how='outer')
    weight_diff.fillna(0, inplace=True)
    weight_diff = weight_diff[(weight_diff['past_weight'] != 0) | (weight_diff['trade_weight'] != 0)]  # 有交易的trade_target
    weight_diff = pd.merge(weight_diff, today_stock_return, on=['Symbol'], how='left')
    weight_diff['diff'] = weight_diff['trade_weight'] - weight_diff['past_weight']
    weight_diff.loc[abs(weight_diff['diff']) < 0.001, 'diff'] = 0  # 差距过小的不进行交易
    weight_diff.loc[weight_diff['trade_weight'] == 0, 'diff'] = \
        -weight_diff.loc[weight_diff['trade_weight'] == 0, 'past_weight']



    sell_list = list(weight_diff.loc[np.where((weight_diff['diff'] < 0)
                                                & (weight_diff['vwap_next'] != 0)), 'Symbol'])
    buy_list = list(weight_diff.loc[np.where((weight_diff['diff'] > 0)
                                                & (weight_diff['vwap_next'] != 0)), 'Symbol'])
    stable_list = [stock_ for stock_ in holding_df.index if stock_ not in sell_list + buy_list]
    stable_list.remove('cash')

    past_value = holding_df['past_value'].sum()  # 前一天持有的所有资产，因为要提前做交易单，所以按前一天净值算
    # 未停牌的能卖出的按开盘价结算卖出
    total_sell = 0


    for stock_ in sell_list:
        stock_info = weight_diff[weight_diff['Symbol'] == stock_]
        next_holding = holding_df.loc[stock_, 'past_value'] / stock_info['vwap'].iloc[0] \
                        * stock_info['vwap_next'].iloc[0]  # 转日开盘后个股价值
        holding_df.loc['cash', 'past_value'] += -next_holding * stock_info['diff'].iloc[0] \
                                                / stock_info['past_weight'].iloc[0] \
                                                * (1 - sell_trade_cost)  # 卖出部分变现
        holding_df.loc[stock_, 'past_value'] = next_holding * stock_info['trade_weight'].iloc[0] \
                                                / stock_info['past_weight'].iloc[0]  # 持仓改变
        total_sell += -next_holding * stock_info['diff'].iloc[0] \
                                                / stock_info['past_weight'].iloc[0] * (1 - sell_trade_cost)
    for stock_ in drop_list:  # 直接被收购退市的，按价值卖出
        holding_df.loc['cash', 'past_value'] += holding_df.loc[stock_, 'past_value'] * (1 - sell_trade_cost)
        holding_df.loc[stock_, 'past_value'] = 0
        total_sell += holding_df.loc[stock_, 'past_value'] * (1 - sell_trade_cost)

    # 剩余的钱且未停牌的按weight重置进行买入
    buy_df = weight_diff[weight_diff['Symbol'].isin(buy_list)]['diff'].sum()
    buy_cash = holding_df.loc['cash'].iloc[0]  # 可以买的钱

    total_buy = 0
    for stock_ in buy_list:
        stock_info = weight_diff[weight_diff['Symbol'] == stock_]
        buy_asset_total = past_value * stock_info['diff'].iloc[0]  # 按总资产计算的购买数量
        buy_asset_cash = buy_cash * stock_info['diff'].iloc[0] / buy_df  # 按现有资金计算的购买量
        buy_asset = min(buy_asset_cash, buy_asset_total)
        holding_df.loc['cash', 'past_value'] -= buy_asset
        if stock_info['past_weight'].iloc[0] == 0:  # 昨日没持仓
            holding_df.loc[stock_, 'past_value'] = buy_asset * (1 - buy_trade_cost)
        else:  # 昨天有持仓
            holding_df.loc[stock_, 'past_value'] = holding_df.loc[stock_, 'past_value'] \
                                                    / stock_info['vwap'].iloc[0] \
                                                    * stock_info['vwap_next'].iloc[0] \
                                                    + buy_asset * (1 - buy_trade_cost)
        total_buy += buy_asset

    # 无操作个股按收盘、开盘结算
    for stock_ in stable_list:
        stock_info = weight_diff[weight_diff['Symbol'] == stock_]
        holding_df.loc[stock_, 'past_value'] = holding_df.loc[stock_, 'past_value'] \
                                                / stock_info['vwap'].iloc[0] \
                                                * stock_info['vwap_next'].iloc[0]  # 转日开盘后个股价值


    # 更新持仓，去除0的
    holding_df = holding_df[holding_df['past_value'] != 0]
    if 'cash' not in holding_df.index:
        holding_df.loc['cash'] = 0
    # holding_df.dropna(inplace=True)

    # 尾盘结算价格
    holding_df.reset_index(inplace=True)

    holding_df.set_index('Symbol', inplace=True)
    turnover_rate = (total_buy + total_sell) / past_value / 2

    return holding_df, turnover_rate



In [2]:

startdate = '2019-01-01'
enddate = '2023-12-31'
chosen_index = '000905.SH'
barra_limit = 0.3

# get prediction
score_df = pd.read_parquet('prediction_xgb_outsample.parquet')
score_df.reset_index(inplace=True)

# get return data
return_df = get_price([],startdate, enddate)
return_df = return_df.stack().reset_index().rename(columns={0:'vwap'})
return_df['vwap_next'] = return_df.groupby('Symbol')['vwap'].shift(-1)

# get barra and industrial data
barra_df = pd.read_parquet(os.path.join(fc.BARR_DIR, 'BarraFactorTable.parquet'))
barra_df['Date'] = pd.to_datetime(barra_df['Date'])


# merge barra data to return
return_df = pd.merge(return_df, barra_df, on=['Date', 'Symbol'], how='left')

# merge barra, industrial and return data to score
score_df = pd.merge(score_df, return_df, on=['Date', 'Symbol'], how='left')
score_df.dropna(inplace=True)

# get index weight
index_weight = pd.read_parquet(os.path.join(fc.BARR_DIR, 'idx__csi500_weight.parquet'))
index_weight = index_weight[['Date','Symbol', 'Weight']]
index_weight['Date'] = pd.to_datetime(index_weight['Date'])
index_weight = pd.merge(index_weight, return_df, on=['Date', 'Symbol'], how='left')
index_weight.dropna(inplace=True)

# get index return
index_return = pd.read_csv(f'{chosen_index}.csv')[['ts_code', 'trade_date', 'close']]
index_return['trade_date'] = pd.to_datetime(index_return['trade_date'], format='%Y%m%d')
index_return = index_return.loc[(index_return.trade_date >= startdate) & (index_return.trade_date <= enddate), ['ts_code','trade_date', 'close']]
index_return.columns = ['index', 'Date', 'index_close']

# define barra and industry list
barra_list = ['Beta', 'Momentum', 'Size', 'EarningsYield', 'ResidualVolatility', 'Growth',
    'BooktoPrice', 'Leverage', 'Liquidity', 'NonlinearSize']

barra_std_list = [barra_ + '_std' for barra_ in barra_list]

industry_list = ['Bank',
    'RealEstate', 'Pharmaceutical', 'CateringTourism',
    'CommercialRetailing', 'Mechanical', 'ConstructionMaterials',
    'HouseholdAppliances', 'TextileApparel', 'FoodBeverage',
    'ElectronicComponents', 'Automobile', 'LightManufacturing',
    'PowerUtilities', 'Comprehensive', 'Communications', 'Others',
    'PetroleumPetrochemical', 'NonferrousMetals', 'AgricultureFarming',
    'Architecture', 'Computers', 'Transport', 'BasicChemicals', 'Coal',
    'ElectricalEquipment', 'DefenseMilitary', 'NonBankFinance', 'Steel',
    'Media']

# get index barra and industry
df_index = pd.DataFrame(columns=barra_list + industry_list + barra_std_list)
for _date, _df in index_weight.groupby('Date'):
    # average index barra
    df_index.loc[_date, barra_list] = np.average(_df[barra_list], weights=_df['Weight'], axis=0)
#     # average index barra std
#     df_index.loc[_date, barra_std_list] = np.average(
#             (_df[barra_list] - df_index.loc[_date, barra_list]) ** 2, weights=_df['Weight'],
#             axis=0) ** 0.5
    # industry weight in index
    df_index.loc[_date, industry_list] = np.average(_df[industry_list], weights=_df['Weight'], axis=0)

# calculate barra limit
for _barra in barra_list:
    df_index[_barra + '_lower'] = df_index[_barra] - barra_limit
    df_index[_barra + '_upper'] = df_index[_barra] + barra_limit

# only consider stocks in index
score_df = pd.merge(score_df, index_weight[['Date', 'Symbol', 'Weight']], on=['Date', 'Symbol'], how = 'left')
score_df = score_df[score_df['Weight'].notna()]




In [3]:
date_list = score_df['Date'].drop_duplicates().sort_values().to_list()
opt_weight = {}  # 字典形式储存结果
initial_asset = 1000
asset_record = pd.DataFrame(columns=['asset'])  # 纪录每日净值
holding_record = {}  # 持仓权重，尾盘更新
holding_record[date_list[0]] = pd.DataFrame({'Symbol':['cash'],
                                                'past_value':[initial_asset]}).set_index('Symbol')

# initial record dataframe
turnover_record = pd.DataFrame(columns=['turnover_rate'])  # 纪录换手率
holding_value_record = pd.DataFrame(columns=['num', 'value'])  # 记录持仓数量和平均市值
# 记录Barra持仓
barra_daily_record = {
    'Beta': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'Momentum': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'Size': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'EarningsYield': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'ResidualVolatility': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'Growth': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'BooktoPrice': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'Leverage': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'Liquidity': pd.DataFrame(columns=['index', 'diff', 'barra']),
    'NonlinearSize': pd.DataFrame(columns=['index', 'diff', 'barra'])
}


In [6]:
for date_num in tqdm(range(0, len(date_list) - 1)):
    _date = date_list[date_num]
    next_date = date_list[date_num + 1]
    stock_today = score_df[score_df['Date'] == _date]
    stock_today.dropna(inplace=True)
    index_today = df_index.loc[_date]
    holding_df = holding_record[_date]

    past_weight = holding_df.reset_index()
    past_weight['past_value'] = past_weight['past_value'] / past_weight['past_value'].sum()
    past_weight.columns = ['Symbol', 'past_weight']
    past_weight = past_weight[past_weight['Symbol'] != 'cash']
    past_weight = pd.merge(left=stock_today[['Symbol']], right=past_weight,
                                on='Symbol', how='outer')
    past_weight.fillna(0, inplace=True)
    stock_today = pd.merge(left=stock_today, right=past_weight[['Symbol']],
                                on='Symbol', how='outer')
    stock_today['Date'] = _date


    barra_limit_tmp = barra_limit
    # 可能有突然被收购股
    nan_df = stock_today[pd.isna(stock_today['pred'])]
    if len(nan_df) > 0:
        drop_list = list(nan_df['Symbol'])
        stock_today_drop = stock_today[~stock_today['Symbol'].isin(drop_list)]
        past_weight_drop = past_weight[~past_weight['Symbol'].isin(drop_list)]

        try:
            opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight_drop,
                                                    stock_today_drop, index_today, barra_limit_tmp, turnover=0.15)
        except:
            try:
                opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight_drop,
                                                        stock_today_drop, index_today, barra_limit_tmp, turnover=0.2)
            except:
                    barra_limit_tmp += barra_limit/2
                    opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight_drop,
                                                         stock_today_drop, index_today, barra_limit_tmp, turnover=0.2)
            turnover = 0.2
        while status not in ['optimal', 'optimal_inaccurate']:
            turnover += 0.05
            opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight_drop,
                                                    stock_today_drop, index_today, barra_limit_tmp, turnover=turnover)
        # 进行交易
        trade_target = pd.DataFrame(opt_weight[_date].values(), index=opt_weight[_date].keys()).reset_index()
        trade_target.columns = ['Symbol', 'trade_weight']
        nan_df = pd.DataFrame({'Symbol':drop_list})
        nan_df['trade_weight'] = 0
        trade_target = pd.concat([trade_target, nan_df], axis=0)
        today_stock_return = return_df[return_df['Date'] == _date][['Date', 'Symbol', 'vwap', 'vwap_next']]
        holding_df, turnover_rate = trade_cal(past_weight, holding_df, trade_target, today_stock_return, sell_trade_cost=0.0005, buy_trade_cost=0.0005, drop_list = drop_list)   

    else:
        try:
            opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight,
                                                    stock_today, index_today, barra_limit_tmp, turnover=0.15)
        except:
            try:
                opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight,
                                                            stock_today, index_today, barra_limit_tmp, turnover=0.2)
            except:
                    barra_limit_tmp += barra_limit/2
                    opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight,
                                                         stock_today, index_today, barra_limit_tmp, turnover=0.2)
            turnover = 0.2
        while status not in ['optimal', 'optimal_inaccurate']:
            turnover += 0.05
            opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight,
                                                    stock_today, index_today, barra_limit_tmp, turnover=turnover)
            
        trade_target = pd.DataFrame(opt_weight[_date].values(), index=opt_weight[_date].keys()).reset_index()
        trade_target.columns = ['Symbol', 'trade_weight']
        today_stock_return = return_df[return_df['Date'] == _date][['Date', 'Symbol', 'vwap', 'vwap_next']]

        holding_df, turnover_rate = trade_cal(past_weight, holding_df, trade_target, today_stock_return, sell_trade_cost=0.0005, buy_trade_cost=0.0005)

    # 净值和持仓记录
    holding_record[next_date] = holding_df
    asset_record.loc[next_date] = holding_df['past_value'].sum() / initial_asset
    turnover_record.loc[next_date] = turnover_rate
    today_mkt = pd.DataFrame(holding_df).reset_index()
    today_mkt['Date'] = next_date

    # 计算barra暴露
    today_barra = pd.DataFrame(holding_df).reset_index()
    today_barra['Date'] = next_date
    today_barra_all = barra_df[barra_df['Date'] == next_date]
    today_barra = pd.merge(left=today_barra, right=today_barra_all, on=['Symbol', 'Date'], how='inner')
    today_barra[barra_list] = today_barra[barra_list] * today_barra['past_value'].values.reshape(-1, 1) \
                                / today_mkt['past_value'].sum()
    index_next = df_index.loc[next_date]

    barra_list
    for _barr in barra_list:
        diff = today_barra[_barr].sum() - index_next[_barr]
        barra_daily_record[_barr].loc[next_date] = [index_next[_barr], diff, today_barra[_barr].sum()]

 63%|██████▎   | 769/1212 [01:10<00:40, 10.90it/s]


TypeError: 'NoneType' object is not subscriptable

In [7]:
date_num = 769
_date = date_list[date_num]
next_date = date_list[date_num + 1]
stock_today = score_df[score_df['Date'] == _date]
stock_today.dropna(inplace=True)
index_today = df_index.loc[_date]
holding_df = holding_record[_date]

past_weight = holding_df.reset_index()
past_weight['past_value'] = past_weight['past_value'] / past_weight['past_value'].sum()
past_weight.columns = ['Symbol', 'past_weight']
past_weight = past_weight[past_weight['Symbol'] != 'cash']
past_weight = pd.merge(left=stock_today[['Symbol']], right=past_weight,
                            on='Symbol', how='outer')
past_weight.fillna(0, inplace=True)
stock_today = pd.merge(left=stock_today, right=past_weight[['Symbol']],
                            on='Symbol', how='outer')
stock_today['Date'] = _date



In [8]:
stock_today

,Date,Symbol,pred,vwap,vwap_next,Country,Beta,Momentum,Size,EarningsYield,...,Computers,Transport,BasicChemicals,Coal,ElectricalEquipment,DefenseMilitary,NonBankFinance,Steel,Media,Weight
0,2022-03-07,000009.SZ,0.000592,111.601591,107.713274,1.0,1.995697,-0.179439,-0.510411,-0.556621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005239
1,2022-03-07,000012.SZ,0.000592,219.120260,210.605061,1.0,0.287828,0.205085,-0.747446,0.816485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001765
2,2022-03-07,000021.SZ,0.000592,179.555586,173.944727,1.0,-0.035284,-1.368878,-0.957911,-0.391310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001890
3,2022-03-07,000027.SZ,0.000592,120.479419,115.929185,1.0,-0.118033,-0.469767,-0.459731,0.232949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002107
4,2022-03-07,000028.SZ,0.000592,140.850721,135.292241,1.0,-1.172088,-0.883761,-1.213396,0.907019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2022-03-07,688208.SH,0.000592,46.831686,45.185303,1.0,-0.072813,-0.806047,-0.914620,-0.816844,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002000
496,2022-03-07,688289.SH,0.000592,48.164531,45.803241,1.0,-1.107701,-2.211599,-1.003799,0.702666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000885
497,2022-03-07,688321.SH,0.000592,27.479515,26.585328,1.0,-0.389968,-1.699728,-1.469852,-1.309147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001092
498,2022-03-07,688521.SH,0.000592,56.543705,57.161000,1.0,1.077729,-1.499451,-0.633149,-1.401276,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002248


In [ ]:
past_weight = past_weight_drop
stock_today = stock_today_drop
period_list = ['pred']
turnover = 0.15
barra_limit = 0.3

# 定义要优化的变量，设置非负约束
weights = cp.Variable(shape=(len(stock_today), 1), nonneg=True)

# 收集约束条件
constraints = []

# 1. 权重之和等于1
constraints.append(cp.sum(weights) == 1)

# 2. 个股权重限制为1%
constraints.append(weights <= 0.01)

# 3. 行业权重限制与指数的差异限制为5%
constraints.append(weights[:, 0] @ stock_today[industry_list] - index_today[industry_list] <= 0.05)
constraints.append(weights[:, 0] @ stock_today[industry_list] - index_today[industry_list] >= -0.05)

# 4. barra 偏离限制
for _barra in barra_list:
    constraints.append(weights[:, 0] @ stock_today[_barra] - index_today[_barra] <= barra_limit)
    constraints.append(weights[:, 0] @ stock_today[_barra] - index_today[_barra] >= -barra_limit)

# 5. 限制换手率（权重变化的总和）为15%
trade_cost_sum = cp.sum(cp.abs(weights - past_weight[['past_weight']].iloc[:len(stock_today)]))

if past_weight['past_weight'].sum() > 0.5:  # 首日排除
    constraints.append(trade_cost_sum <= turnover)  # 单日换手不超过15%

# 定义目标函数
profit_sum = cp.sum(cp.multiply(weights, stock_today[period_list])) 
obj = profit_sum 

# 传递问题给求解器
problem = cp.Problem(cp.Maximize(obj), constraints)


try:
    problem.solve(solver='ECOS', qcp=True, max_iters=2000)
except:
    problem.solve()

weights_value = weights.value
weights_value = pd.concat([stock_today[['Symbol']].reset_index(drop=True),
                            pd.DataFrame(weights_value[:, 0], columns=['weight']).reset_index(drop=True)],
                            axis=1)
weights_value.loc[weights_value['weight'] < 0.00001, 'weight'] = 0
weights_value = weights_value[weights_value['weight'] != 0]
weights_value['weight'] = weights_value['weight'] / weights_value['weight'].sum()
weights_value = pd.merge(left=weights_value, right=past_weight[['Symbol']], on='Symbol', how='outer')
weights_value.fillna(0, inplace=True)
weights_value = weights_value.set_index('Symbol')['weight'].to_dict()
weights_value


In [ ]:





# optimize
opt_weight[_date], status = port_opt(['pred'], industry_list, barra_list, past_weight,
                                                    stock_today, index_today, barra_limit, turnover=0.15)

trade_target = pd.DataFrame(opt_weight[_date].values(), index=opt_weight[_date].keys()).reset_index()
trade_target.columns = ['Symbol', 'trade_weight']
today_stock_return = return_df[return_df['Date'] == _date][['Date', 'Symbol', 'vwap', 'vwap_next']]

holding_df, turnover_rate = trade_cal(past_weight, holding_df, trade_target, today_stock_return)

# 净值和持仓记录
holding_record[next_date] = holding_df
asset_record.loc[next_date] = holding_df['past_value'].sum() / initial_asset
turnover_record.loc[next_date] = turnover_rate
today_mkt = pd.DataFrame(holding_df).reset_index()
today_mkt['Date'] = next_date

# 计算barra暴露
today_barra = pd.DataFrame(holding_df).reset_index()
today_barra['Date'] = next_date
today_barra_all = barra_df[barra_df['Date'] == next_date]
today_barra = pd.merge(left=today_barra, right=today_barra_all, on=['Symbol', 'Date'], how='inner')
today_barra[barra_list] = today_barra[barra_list] * today_barra['past_value'].values.reshape(-1, 1) \
                            / today_mkt['past_value'].sum()
index_next = df_index.loc[next_date]

barra_list
for _barr in barra_list:
    diff = today_barra[_barr].sum() - index_next[_barr]
    barra_daily_record[_barr].loc[next_date] = [index_next[_barr], diff, today_barra[_barr].sum()]